In [ ]:
!pip install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=f85149659eda5d9e8c47dae8450f05b4413ac44e92a8e44eabbc177334a6008a
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2022-06-21 07:09:23--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   116MB/s    in 0.2s    

2022-06

In [ ]:
import pandas as pd
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
import numpy as np
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
data=pd.read_excel("/Data/data.xlsx")
X=list(data['text'])
y=list(data['Cấp 2'])

In [ ]:
def standard_data(data):
    for id in range(len(data)):
        data[id] = re.sub(r"[\.,\?]+$-", "", data[id])
        data[id] = data[id].replace(",", " ").replace(".", " ") \
            .replace(";", " ").replace("“", " ") \
            .replace(":", " ").replace("”", " ") \
            .replace('"', " ").replace("'", " ") \
            .replace("!", " ").replace("?", " ") \
            .replace("-", " ").replace("|", " ")
        data[id] = data[id].strip().lower()
        data[id] = re.sub(r'\s\s+', ' ', data[id])
    return data

In [ ]:
def remove_stopword(texts,stop_word):
  data=[]
  for i in tqdm(range(len(texts))):
    data.append(" ".join([word for word in texts[i].split() if word not in stop_word]))
  return data
with open("/Data/stopword.txt",encoding="UTF-8") as f:
    stop_word=f.read().splitlines()

In [ ]:
def syllable(texts):
  text=[]
  for line in texts:
    segmen=rdrsegmenter.tokenize(line)
    segmen = ' '.join([' '.join(x) for x in segmen])
    text.append(segmen)
  return text

In [ ]:
y=list(y)
X=standard_data(X)
X=remove_stopword(X,stop_word)

100%|██████████| 7217/7217 [01:23<00:00, 86.83it/s]


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
with open("/Data/viettel_train_input_no_tokenize.txt","w",encoding='UTF-8') as f:
  for line in X_train:
    f.write(line+"\n")
with open("/Data/viettel_train_label.txt","w",encoding='UTF-8') as f:
    for line in y_train:
      f.write(line+"\n")
with open("/Data/viettel_test_input_no_tokenize.txt","w",encoding='UTF-8') as f:
    for line in X_test:
      f.write(line+"\n")
with open("/Data/viettel_test_label.txt","w",encoding='UTF-8') as f:
    for line in y_test:
      f.write(line+"\n")

In [ ]:
X_train=syllable(X_train)
X_test=syllable(X_test)

In [ ]:
with open("/Data/viettel_train_input.txt","w",encoding='UTF-8') as f:
  for line in X_train:
    f.write(line+"\n")
with open("/Data/viettel_test_input.txt","w",encoding='UTF-8') as f:
  for line in X_test:
    f.write(line+"\n")